# Granite simulations

<p class="acknowledgement">Written by Svetlana Kyas (ETH Zurich) on Feb 4th, 2022</p>

This tutorial demonstrates how Reaktoro can be used for performing equilibrium simulations for the granite rock mixed
with NaCl-brine. We use `aq17` thermofun database.

```{note}
If your main interest is in computing thermodynamic properties, rather than chemical equilibrium and
kinetics modeling, you may want to check [ThermoFun](https://thermohub.org/thermofun/thermofun/),
which is an excellent project dedicated to this task.

This tutorial is joint work with Dan Miron, PSI.
```

In [ ]:
# Import reaktoro package
from reaktoro import *

# Define Thermofun database
db = ThermoFunDatabase("aq17")

We define an aqueous solution with concrete species' list and set up the Debye-Huckel activity model with
provided parameters. The minerals' assembly includes 12 minerals. These two phases will define the chemical system.

In [ ]:
solution = AqueousPhase("H2O@ H+ OH- Cl- HCl@ Na+ NaOH@ NaHSiO3@ NaCl@ NaAl(OH)4@ SiO2@ HSiO3- "
                            "K+ KOH@ KCl@ KAlO2@ Al+3 AlOH+2 Al(OH)2+ Al(OH)3@ Al(OH)4-")

# Set up a and b parameters for ionic species (NaCl, b = 0.064, a = 3.72)
params = ActivityModelDebyeHuckelParams()
params.aiondefault = 3.72
params.biondefault = 0.064
params.bneutraldefault = 0.064
solution.setActivityModel(ActivityModelDebyeHuckel(params))

# Define minerals phases
minerals = MineralPhases("Quartz Diaspore Gibbsite Andalusite Kyanite "
                             "Sillimanite Muscovite Paragonite Pyrophyllite "
                             "Kaolinite Albite Microcline")

# Define chemical system by providing database, aqueous phase, and minerals
system = ChemicalSystem(db, solution, minerals)

With the code below, we set up equilibrium specifications, equilibrium conditions, and equilibrium solver used for the
equilibrium calculations:

In [ ]:
# Specify conditions to be satisfied at chemical equilibrium
specs = EquilibriumSpecs(system)
specs.temperature()
specs.pressure()

# Define temperature and pressure
T = 400.0 # in Celsius
P = 1e3 # in bar

# Define conditions to be satisfied at chemical equilibrium
conditions = EquilibriumConditions(specs)
conditions.temperature(T, "celsius")
conditions.pressure(P, "bar")

# Define equilibrium solver
solver = EquilibriumSolver(specs)

In the initial chemical state, we mix 100 g of granite and 20 g of NaCl-brine (1 molal NaCl concentration).
Granite rock is composed of 35% of quartz, 19% muscovite, 29% of albite, and 17% of microcline. This corresponds to
the mass of the minerals we set below. Finally, we run equilibrium calculations and create chemical and aqueous
properties instances based on the obtained chemical state.

In [ ]:
# Define initial equilibrium state
state = ChemicalState(system)
# Define initial solution amount (1 molal NaCl-brine)
state.set("H2O@" , 20, "g")
state.set("NaCl@", 0.02, "mol")
# Define initial composition of granite (100g)
state.set("Quartz"    , 35, "g") # 35% of granite
state.set("Microcline", 17, "g") # 17% of granite
state.set("Albite"    , 29, "g") # 29% of granite
state.set("Muscovite" , 19, "g") # 19% of granite

# Equilibrate the initial state with given conditions and component amounts
res = solver.solve(state, conditions)
print("res (aq17) = ", res.optima.succeeded)

# Output the chemical state to a console
print(state)

# Create chemical properties instance and output it to a console
props = ChemicalProps (state)
print(props)

# Create aqueous properties instance and output it to a console
aprops = AqueousProps (state)
print(aprops)